In [2]:
import os
from secrets import base_path

paths_dict =  {
    "zenodo_files_results": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "zenodo_files_results.json"),
    "gzoltar_execution_path": os.path.join(base_path, "gzoltar_files", ),
    "defects4j_path": os.path.join(base_path, "gzoltar_files", "defects4j"),
    "gzoltar_path": os.path.join(base_path, "gzoltar_files", "gzoltar"),
    "bugs_data": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "bug_reports_with_stack_traces_details.json"),
    "current_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "code", "auxiliary-scripts"),
    "output_folder": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "gzoltar_files"),
}

files_to_copy_per_bug = {
    "Cli": {
        "default": ".defects4j.config defects4j.build.properties build.xml pom.xml maven.xml",
        "5": ".defects4j.config defects4j.build.properties build.xml gump.xml maven.xml"
    },
    "Closure":{
        "default": ".defects4j.config defects4j.build.properties build.xml",
        "125": ".defects4j.config defects4j.build.properties build.xml closure-compiler.pom",
    },
    "Codec":{
        "default": ".defects4j.config defects4j.build.properties build.xml default.properties pom.xml",
    },
    "Collections":{
        "default": ".defects4j.config defects4j.build.properties build.xml default.properties pom.xml",
    },
    "Compress":{
        "default": ".defects4j.config defects4j.build.properties build.xml pom.xml maven-build.xml maven-build.properties",
    },
    "Csv":{
        "default": ".defects4j.config defects4j.build.properties build.xml pom.xml maven-build.xml maven-build.properties",
    },
    "Gson":{
        "default": ".defects4j.config defects4j.build.properties pom.xml gson/build.xml gson/pom.xml gson/maven-build.xml gson/maven-build.properties",
    },
    "JacksonCore":{
        "default": ".defects4j.config defects4j.build.properties pom.xml build.xml maven-build.xml maven-build.properties",
    },
    "JacksonDatabind":{
        "default": ".defects4j.config defects4j.build.properties pom.xml build.xml maven-build.xml maven-build.properties",
    },
    "Jsoup":{
        "default": ".defects4j.config defects4j.build.properties pom.xml build.xml maven-build.xml maven-build.properties",
    },
    "JxPath":{
        "default": ".defects4j.config defects4j.build.properties build.xml project.properties project.xml",
    },
    "Lang":{
        "default": ".defects4j.config defects4j.build.properties pom.xml build.xml maven-build.xml maven-build.properties",
        "6": ".defects4j.config defects4j.build.properties pom.xml build.xml default.properties",
        "61": ".defects4j.config defects4j.build.properties build.xml maven.xml project.properties project.xml",
    },
    "Math":{
        "default": ".defects4j.config defects4j.build.properties pom.xml build.xml",
    },
    "Mockito":{
        "default": ".defects4j.config defects4j.build.properties build-ant.xml build.gradle gradle.properties settings.gradle version.properties",
    },
    "Time":{
        "default": ".defects4j.config defects4j.build.properties build.xml maven-build.properties maven-build.xml pom.xml",
        "14": ".defects4j.config defects4j.build.properties pom.xml build.xml"
    }
}

java_version_for_defects4j_execution = "1.8.0_292" # Change according to necessary


# Coming back to the project's folder
os.chdir(paths_dict["current_path"])

In [14]:
%run ../utils.py

import subprocess
import sys
import time
import os

zenodo_files_results = json_file_to_dict(paths_dict["zenodo_files_results"])
bugs_data = json_file_to_dict(paths_dict["bugs_data"])
projects_already_done = [ "Cli", "Closure", "Codec", "Collections", "Compress", "Csv", "Gson", "JacksonCore", "Jsoup", "JxPath", "Lang", "Mockito", "Time"]
already_done = ["JacksonDatabind_14", "JacksonDatabind_15", "JacksonDatabind_25", "JacksonDatabind_37", "JacksonDatabind_73", "JacksonDatabind_9", "Math_79", "Math_81"]
for bug_report in zenodo_files_results["bugs_not_found"]:
    if bug_report in already_done:
        continue

    print("Done")

    project = bug_report.split("_")[0]
    bug_id = bug_report.split("_")[1]

    if project in projects_already_done:
        continue

    if bug_report in ["JacksonDatabind_104", "JacksonDatabind_17", "JacksonDatabind_28", "JacksonDatabind_32", "JacksonDatabind_59", "JacksonDatabind_61", "JacksonDatabind_62", "JacksonDatabind_65", "JacksonDatabind_68", "JacksonDatabind_69", "JacksonDatabind_70", "JacksonDatabind_72", "JacksonDatabind_90", "JacksonDatabind_91", "JacksonDatabind_93", "JacksonDatabind_95"]:
        continue #TODO: Solve this problem later

    if bug_report in ["Math_49", "Math_58", "Math_85"]:
        continue #TODO: Math 49 did not finish running in more than 12 hours

    print("************")
    print(bug_report)

    bug_report_commit_hash = bugs_data[project][bug_id]["bug_report_commit_hash"]

    defects4j_buggy_folder = os.path.join(paths_dict["gzoltar_execution_path"], bug_report + "_buggy")
    defects4j_copy_folder = os.path.join(paths_dict["gzoltar_execution_path"], bug_report + "_copy")

    # Configuring the environment variables
    # 1. Java
    java_home = subprocess.check_output(["/usr/libexec/java_home", "-v", java_version_for_defects4j_execution]).decode().strip() # Making sure the current Java version is 1.8 (defects4j requirement)
    os.environ['JAVA_HOME'] = java_home
    # 2. defects4j
    # os.environ["PATH"] += os.pathsep + paths_dict['defects4j_path'] + "/framework/bin"
    os.environ["PATH"] = """/Users/lorenapacheco/opt/anaconda3/bin:/Users/lorenapacheco/opt/anaconda3/condabin:/Users/lorenapacheco/.pyenv/shims:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/lorenapacheco/Library/Python/3.8/bin:/Library/Apple/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Applications/Visual Studio Code.app/Contents/Resources/app/bin:/Users/lorenapacheco/Library/Application Support/JetBrains/Toolbox/scripts:/Users/lorenapacheco/Concordia/Masters/defects4j/framework/bin"""
    os.environ["D4J_HOME"] = paths_dict['defects4j_path']
    os.environ["TZ"] = 'America/Los_Angeles'
    os.environ["LC_ALL"] = "en_US.UTF-8"
    os.environ["LANG"] = "en_US.UTF-8"
    os.environ["LANGUAGE"] = "en_US.UTF-8"
    # 3. gzoltar
    os.environ['GZOLTAR_AGENT_JAR'] = paths_dict['gzoltar_path'] + "/com.gzoltar.agent.rt/target/com.gzoltar.agent.rt-1.7.3-SNAPSHOT-all.jar"
    os.environ['GZOLTAR_CLI_JAR'] = paths_dict['gzoltar_path'] + "/com.gzoltar.cli/target/com.gzoltar.cli-1.7.3-SNAPSHOT-jar-with-dependencies.jar"

    # ----- defects4j part -----

    # Running defects4j checkout
    defects4j_checkout_cmd = f"defects4j checkout -p {project} -v {bug_id}b -w {defects4j_buggy_folder}"
    print("> " + defects4j_checkout_cmd)
    subprocess.run(defects4j_checkout_cmd, shell=True, check=True)

    # Creating a copy of the folder
    copy_folder_cmd = f"cp -r {defects4j_buggy_folder} {defects4j_copy_folder}"
    subprocess.run(copy_folder_cmd, shell=True, check=True)

    # Checkout in the bug report commit in the copy folder
    checkout_commit_cmd = f"git checkout {bug_report_commit_hash}"
    subprocess.run(checkout_commit_cmd, shell=True, check=True, cwd=defects4j_copy_folder)

    # Copying defects4j important files to the copy folder
    files_to_copy = files_to_copy_per_bug[project].get(bug_id, files_to_copy_per_bug[project]['default'])
    for file in files_to_copy.split(" "):
        if "/" in file:
            last_slash_index = file.rindex("/")
            folder = file[:last_slash_index+1]
            copy_files_cmd = f"cp {file} {defects4j_copy_folder}/{folder}"
        else:
            copy_files_cmd = f"cp {file} {defects4j_copy_folder}"
        subprocess.run(copy_files_cmd, shell=True, check=True, cwd=defects4j_buggy_folder)

    cmd = "echo 'org.apache.commons.**.*' > all-classes.txt"
    subprocess.run(cmd, shell=True, check=True, cwd=defects4j_copy_folder)


    if bug_report == "Cli_14":
        remove_non_utf8_characters(defects4j_copy_folder + "/src/test/org/apache/commons/cli2/bug/BugLoopingOptionLookAlikeTest.java")

    if project == "JacksonCore":
        folder = os.path.join(defects4j_copy_folder, "src", "main", "java", "com", "fasterxml", "jackson", "core", "json")
        filename = "PackageVersion"
        # The content for the Java file
        content = """
package com.fasterxml.jackson.core.json;

import com.fasterxml.jackson.core.Version;
import com.fasterxml.jackson.core.Versioned;
import com.fasterxml.jackson.core.util.VersionUtil;

/**
 * Automatically generated from PackageVersion.java.in during
 * packageVersion-generate execution of maven-replacer-plugin in
 * pom.xml.
 */
public final class PackageVersion implements Versioned {
    public final static Version VERSION = VersionUtil.parseVersion(
        "2.6.1-SNAPSHOT", "com.fasterxml.jackson.core", "jackson-core");

    @Override
    public Version version() {
        return VERSION;
    }
}
"""
        create_java_file(folder, filename, content)
    elif project == "JacksonDatabind":
        folder = os.path.join(defects4j_copy_folder, "src", "main", "java", "com", "fasterxml", "jackson", "databind", "cfg")
        filename = "PackageVersion"
        # The content for the Java file
        content = """
package com.fasterxml.jackson.databind.cfg;

import com.fasterxml.jackson.core.Version;
import com.fasterxml.jackson.core.Versioned;
import com.fasterxml.jackson.core.util.VersionUtil;

/**
 * Automatically generated from PackageVersion.java.in during
 * packageVersion-generate execution of maven-replacer-plugin in
 * pom.xml.
 */
public final class PackageVersion implements Versioned {
    public final static Version VERSION = VersionUtil.parseVersion(
        "2.5.3-SNAPSHOT", "com.fasterxml.jackson.core", "jackson-databind");

    @Override
    public Version version() {
        return VERSION;
    }
}

"""
        create_java_file(folder, filename, content)
        print("OK")
    elif bug_report in ["Math_49", "Math_58", "Math_85"]:
        file_name = os.path.join(defects4j_copy_folder, "src", "test", "java", "org", "apache", "commons", "math", "util", "FastMathStrictComparisonTest.java")
        comment_java_test(file_name,"test1")

    # Defects4j compile
    defects4j_compile_cmd = "defects4j compile"
    print("> " + defects4j_compile_cmd)
    subprocess.run(defects4j_compile_cmd, shell=True, check=True, cwd=defects4j_copy_folder)

    # ---- Gzoltar part ------

    # Navigate to the directory
    os.chdir(defects4j_copy_folder)

    # Get test classpath
    test_classpath = subprocess.check_output(["defects4j", "export", "-p", "cp.test"]).decode().strip()
    # Defects4j test
    defects4j_test_cmd = "defects4j test"
    print("> " + defects4j_test_cmd)
    subprocess.run(defects4j_test_cmd, shell=True, check=True, cwd=defects4j_copy_folder)

    # Get source classes directory
    src_classes_dir_output = subprocess.check_output(["defects4j", "export", "-p", "dir.bin.classes"]).decode().strip()
    src_classes_dir = os.path.join(defects4j_copy_folder, src_classes_dir_output)

    # Get test classes directory
    test_classes_dir_output = subprocess.check_output(["defects4j", "export", "-p", "dir.bin.tests"]).decode().strip()
    test_classes_dir = os.path.join(defects4j_copy_folder, test_classes_dir_output)

    print(f"{project}_{bug_id}_copy's classpath: {test_classpath}")
    print(f"{project}_{bug_id}_copy's bin dir: {src_classes_dir}")
    print(f"{project}_{bug_id}_copy's test bin dir: {test_classes_dir}")

    # Unit tests file name
    unit_tests_file=f"{defects4j_copy_folder}/unit_tests.txt"

    # Gzoltar listTestMethods
    #java_cmd = [
    #    "java",
    #    "-cp",
    #    f"{test_classpath}:{test_classes_dir}:{paths_dict['defects4j_path']}/framework/projects/lib/junit-4.11.jar:{os.environ['GZOLTAR_CLI_JAR']}",
    #    "com.gzoltar.cli.Main",
    #    "listTestMethods",
    #    test_classes_dir,
    #    "--outputFile",
    #    unit_tests_file
    #]

    java_cmd = [
        "java",
        "-cp",
        f"{paths_dict['defects4j_path']}/framework/projects/lib/junit-4.11.jar:{test_classpath}:{test_classes_dir}:{os.environ['GZOLTAR_CLI_JAR']}",
        "com.gzoltar.cli.Main",
        "listTestMethods",
        test_classes_dir,
        "--outputFile",
        unit_tests_file
    ]

    subprocess.run(java_cmd, check=True)

    # Printing the 5 first lines from unit_tests_file
    with open(unit_tests_file, "r") as file:
        for i in range(5):
            line = file.readline().strip()
            print(line)

    loaded_classes_file = os.path.join(paths_dict['defects4j_path'], 'framework', 'projects', project, 'loaded_classes', f'{bug_id}.src')

    with open(loaded_classes_file, 'r') as file:
        loaded_classes = file.read()

    # Getting the classes to debug
    normal_classes = loaded_classes.replace('\n', ':').rstrip(':')
    inner_classes = loaded_classes.replace('\n', '$*:')
    classes_to_debug = f'{normal_classes}{inner_classes}'

    print(f'Likely faulty classes: {classes_to_debug}', file=sys.stderr)

    ser_file = os.path.join(paths_dict['gzoltar_execution_path'], f'{project}_{bug_id}_copy', 'gzoltar.ser')


    # Gzoltar runTestMethods
    command = f'''
    java -XX:MaxPermSize=4096M -javaagent:{os.environ["GZOLTAR_AGENT_JAR"]}=destfile={ser_file},buildlocation={src_classes_dir},inclnolocationclasses=false,output=FILE \
        -cp {src_classes_dir}:{paths_dict["defects4j_path"]}/framework/projects/lib/junit-4.11.jar:{test_classpath}:{os.environ["GZOLTAR_CLI_JAR"]} \
        com.gzoltar.cli.Main runTestMethods \
            --testMethods {unit_tests_file} \
            --collectCoverage
    '''
    subprocess.run(command, check=True, shell=True, executable="/bin/bash")

    # Gzoltar faultLocalizationReport
    command = f'''
    java  -Xmx4G -cp {src_classes_dir}:{paths_dict["defects4j_path"]}/framework/projects/lib/junit-4.11.jar:{test_classpath}:{os.environ["GZOLTAR_CLI_JAR"]} \
        com.gzoltar.cli.Main faultLocalizationReport \
        --buildLocation {src_classes_dir} \
        --granularity line \
        --inclPublicMethods \
        --inclStaticConstructors \
        --inclDeprecatedMethods \
        --dataFile {ser_file} \
        --outputDirectory {defects4j_copy_folder} \
        --family sfl \
        --formula ochiai \
        --metric entropy \
        --formatter txt'''
    subprocess.run(command, check=True, shell=True, executable="/bin/bash")

    #Copying the coverage files to this repo
    coverage_files = ["matrix.txt", "spectra.csv", "statistics.csv", "tests.csv"]
    coverage_files_source_folder = os.path.join(defects4j_copy_folder, "sfl", "txt")
    coverage_files_destination_folder = os.path.join(paths_dict["output_folder"], project, bug_id)

    copy_specific_files(coverage_files, coverage_files_source_folder, coverage_files_destination_folder)

    # Remove the created defects4j folders  to free the space
    delete_folders_cmd = f"""
        rm -rf {defects4j_buggy_folder};
        rm -rf {defects4j_copy_folder}"""
    subprocess.run(delete_folders_cmd, shell=True, check=True)

    # Coming back to the project's folder
    os.chdir(paths_dict["current_path"])

    print("Done")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
